<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/RAG_Personal_Resource_Assistant_Langchain%2C_Cohere_and_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing necessary libraries

Cohere provides free trial keys to use their LLMs. So generate one trial key from dashboard.cohere.com

In [ ]:
!pip install langchain-cohere langchain pdfminer.six chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.2 MB/s eta 0

langchain-cohere: Enables integration of Cohere's language models with LangChain for advanced text generation and processing workflows.

langchain: Provides a modular framework for building language model-powered applications, such as chatbots, question-answering systems, and conversational agents.

pdfminer.six: Facilitates text extraction from PDF files, making it useful for document analysis and preprocessing tasks.

chromadb: A vector database library designed for efficient storage and retrieval of embeddings, ideal for tasks like semantic search and recommendation systems.

## Importing libraries

In [ ]:
import os
from google.colab import userdata
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

## VectorDB setup

In [ ]:
# Define the directory where the Chroma database will persist data
persist_directory = "/content/chroma_db"

# Initialize Cohere embeddings with the specified model
# "embed-english-v3.0" is a pre-trained English language embedding model by Cohere
# The user_agent parameter specifies the tool or library using the Cohere API, in this case, LangChain
embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)


We are processing 2 research papers on transformers and yolo. You can use the PDFs

In [ ]:
# Loop through a list of PDF files to process
for pdf_name in ["/content/1706.03762v7.pdf", "/content/1506.02640v5.pdf"]:
    # Open each PDF file in binary mode
    with open(pdf_name, 'rb') as f:
        # Extract text from the PDF using the extract_text_pdf_miner function
        text = extract_text_pdf_miner(f)

        # Clean the extracted text by removing newline characters and joining into a single string
        cleaned_text = " ".join(text.split("\n"))

        # Initialize a list to store document chunks
        docs = []

        # Create a text splitter to divide the text into manageable chunks
        # Each chunk has a maximum size of 2048 characters with a 512-character overlap
        splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

        # Split the cleaned text into chunks and wrap each chunk in a Document object
        for chunk in splitter.split_text(cleaned_text):
            docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))

    # Create a Chroma collection from the processed documents
    # Use the specified persist directory and embedding model for storage and retrieval
    vector_collection_fixed_size = Chroma.from_documents(
        documents=docs,
        persist_directory=persist_directory,
        embedding=embedding
    )

In [ ]:
# Initialize a Chroma vector database
# The persist_directory specifies the location where the database is stored
# The embedding_function parameter provides the embedding model used for vector representation
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
# Perform a similarity search on the vector database
# The query "What is YOLO?" is used to find the most relevant documents
# k=1 specifies that the top 1 most similar document should be retrieved
# The method also returns relevance scores indicating how closely each document matches the query
vectordb.similarity_search_with_relevance_scores("What is YOLO?", k=1)

[(Document(metadata={'source': '/content/1506.02640v5.pdf'}, page_content='Detection In The Wild  Academic datasets for object detection draw the training and testing data from the same distribution. In real-world applications it is hard to predict all possible use cases and  YOLO is a fast, accurate object detector, making it ideal for computer vision applications. We connect YOLO to a webcam and verify that it maintains real-time performance,  \x0cVOC 2007 AP 59.2 54.2 43.2 36.5 -  Picasso AP Best F1 0.590 53.3 0.226 10.4 0.458 37.8 0.271 17.8 0.051 1.9  People-Art AP 45 26 32  YOLO R-CNN DPM Poselets [2] D&T [4]  (a) Picasso Dataset precision-recall curves.  (b) Quantitative results on the VOC 2007, Picasso, and People-Art Datasets. The Picasso Dataset evaluates on both AP and best F1 score.  Figure 5: Generalization results on Picasso and People-Art datasets.  Figure 6: Qualitative Results. YOLO running on sample artwork and natural images from the internet. It is mostly accurate a

In [ ]:
# Initialize an LLM instance using Cohere's "command-r" model
# The temperature parameter controls randomness in the generated responses; 0 ensures deterministic outputs
llm = ChatCohere(model="command-r", temperature=0)

# Define a prompt template for generating answers based on a given context and question
prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """

# Create a ChatPromptTemplate from the string template, enabling dynamic input for context and question
prompt = ChatPromptTemplate.from_template(prompt_str)

# Create a retrieval pipeline to fetch relevant context and pass through the user's question
retrieval = RunnableParallel(
    {
        # Use the vector database as a retriever to fetch relevant context for the question
        "context": vectordb.as_retriever(),

        # Pass through the user's input question without modification
        "question": RunnablePassthrough()
    }
)

# Define an output parser to format the generated response into a string
output_parser = StrOutputParser()

# Create a processing chain that retrieves context, formats the prompt, generates an LLM response, and parses the output
chain = retrieval | prompt | llm | output_parser

In [ ]:
# Invoke the chain of components (retrieval, prompt generation, LLM processing, and output parsing)
# The question "What is YOLO?" is passed through the chain to generate the response
response = chain.invoke("What is YOLO?")

# Print the response generated by the chain
print(response)

YOLO is a unified model for object detection. It stands for 'You Only Look Once', which refers to the model's simplicity - it only looks at an image once to predict what objects are in the image and where, without the need for a complex pipeline. YOLO is extremely fast and can process streaming video in real-time. It simultaneously predicts multiple bounding boxes and class probabilities, which are then optimised.


Other chain invoking methods!

.invoke(): The goal is to pass in an input and receive the output—neither more nor less.

.batch(): This is faster than using invoke three times when you wish to supply several inputs to get multiple outputs because it handles the parallelization for you.

.stream():  We may begin printing the response before the entire response is complete.

In [ ]:
response_with_batch = chain.batch(["What is Transformers", "How is Transformer different than YOLO?"])

for response in response_with_batch:
  print(response)
  print("\n")

In the context provided, Transformers are sequence transduction models that are based entirely on attention mechanisms, doing away with the recurrent layers normally present in encoder-decoder architectures. They rely on self-attention and allow for more parallelization compared to other models, achieving improved translation quality. The Transformer model architecture consists of stacked self-attention and point-wise, fully connected layers in both the encoder and decoder. This approach enables the model to draw global dependencies between input and output, enhancing its ability to capture dependencies in the data.


While YOLO uses a single convolutional network to simultaneously predict multiple bounding boxes and class probabilities within an image, Transformers rely on self-attention mechanisms to process sequential data. 

YOLO is a real-time object detection system that predicts bounding boxes and their corresponding classes in one forward pass of the network. It's called "You O

In [ ]:
for chunk in chain.stream("What are the 3 vectors in Transformers architecture?"):
  print(chunk, flush=True, end="")

I found no mention of three vectors in the Transformer architecture. However, a key component in the Transformer architecture is the use of three distinct sub-layers in both the encoder and decoder stacks. 

The first sub-layer employs a multi-head self-attention mechanism, which presumably involves multiple attention heads that operate in parallel and attend to different portions of the input sequence. This allows the model to capture different aspects of input dependencies.

The second sub-layer is a simple, position-wise fully connected feed-forward network, which applies the same transformation to all positions in the sequence. 

The third element, not a vector but rather a fundamental aspect of the architecture, is the residual connection applied around each sub-layer. This is followed by layer normalization to stabilize and streamline the information flow. 

These three architectural elements, in combination, form the key building blocks of the Transformer model, enabling it to c